In [13]:
''' 
Overview of what is used here: 

1. setting index to date
2. joinging data by specific columns
3. dropping nan data
4. combining these steps with HOW parameter
5. Renaming column to avoid clash
6. Reading in data from a list of stock csvs with a for loop
7. Creating utility functions to do repeated processes


'''

' \nOverview of what is used here: \n\n1. setting index to date\n2. joinging data by specific columns\n3. dropping nan data\n4. combining these steps with HOW parameter\n5. Renaming column to avoid clash\n6. Reading in data from a list of stock csvs with a for loop\n7. Creating utility functions to do repeated processes\n\n\n'

In [14]:
import pandas as pd
import os

In [3]:
start_date = '2010-01-22'
end_date = '2010-01-26'
dates = pd.date_range(start_date, end_date)
dates[0]

Timestamp('2010-01-22 00:00:00', freq='D')

In [4]:
df1 = pd.DataFrame(index=dates)
df1

""
2010-01-22
2010-01-23
2010-01-24
2010-01-25
2010-01-26


In [5]:
# Key takeaway here is that if you try to join dfSPY with df1 the indices are of different types
# so we have to set dfSPY's indices to dates like we did for ds1
dfSPY = pd.read_csv("SPY.csv", index_col="Date", parse_dates=True, usecols=['Date','Adj Close'],
                    na_values=['nan'])
dfSPY

,Adj Close
Date,
2012-09-12,144.39
2012-09-11,143.91
2012-09-10,143.51
2012-09-07,144.33
2012-09-06,143.77
...,...
2000-02-07,114.65
2000-02-04,114.82
2000-02-03,115.30


In [6]:
# df1 = df1.join(dfSPY)
# # Here we want to drop rows where the value is 'na'
# df1 = df1.dropna()
# df1

In [7]:
# We need to rename the column to the specific stock symbol because below we will encounter clashing 
# column names -> Lecture 10 in this module, 
# 1:10 in vid
dfSPY = dfSPY.rename(columns = {'Adj Close':'SPY'})

In [8]:
df1 = df1.join(dfSPY, how = 'outer')
df1 = df1.dropna()
df1

,SPY
2000-02-01,113.49
2000-02-02,113.58
2000-02-03,115.30
2000-02-04,114.82
2000-02-07,114.65
...,...
2012-09-06,143.77
2012-09-07,144.33
2012-09-10,143.51
2012-09-11,143.91


In [9]:
symbols = ['GOOG', 'IBM', 'GLD']

In [10]:
for symbol in symbols:
    df_temp = pd.read_csv("{}.csv".format(symbol), index_col='Date',
                          parse_dates=True,usecols=['Date','Adj Close'], 
                          na_values =['nan'])
    # we repeat this here to avoid column name clash
    df_temp = df_temp.rename(columns={'Adj Close': symbol})
    df1 = df1.join(df_temp) #use default how = 'left'
#     df1 = df1.dropna(subset=["SPY"])
df1

,SPY,GOOG,IBM,GLD
2000-02-01,113.49,NaN,94.30,NaN
2000-02-02,113.58,NaN,97.30,NaN
2000-02-03,115.30,NaN,100.40,NaN
2000-02-04,114.82,NaN,99.12,NaN
2000-02-07,114.65,NaN,97.83,NaN
...,...,...,...,...
2012-09-06,143.77,699.40,199.10,164.89
2012-09-07,144.33,706.15,199.50,168.44
2012-09-10,143.51,700.77,200.95,167.29
2012-09-11,143.91,692.19,203.27,167.90


In [11]:
# utility function instead of the code above, need to import os (done at the beginning)
# not I am commenting this out below because I have opted to do this notebook within the same directory
def symbol_to_path(symbol, base_dir="/"):
    return os.path.join(base_dir, "{}.csv".format(str(symbol)))


In [12]:
def get_data(symbols, dates):
    df = pd.DataFrame(index=dates)
    if 'SPY' not in symbols:
        symbols.inset(0, 'SPY')
    
    for symbol in symbols:
        path = symbol_to_path(symbol)
        df_temp = pd.read_csv(path, index_col='Date',
                          parse_dates=True,usecols=['Date','Adj Close'], 
                          na_values =['nan'])
        # we repeat this here to avoid column name clash
        df_temp = df_temp.rename(columns={'Adj Close': symbol})
        df = df.join(df_temp) #use default how = 'left'
        df = df.dropna(subset=["SPY"])
